# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = list(zip(zedges[:-1], zedges[1:]))
nbars = [1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985]  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import (BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles)
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

setup_logging()

## Let's compare simple isotropic BAO forecasts against some reference

### SN-weighted version

In [3]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')  # prescription for linear bias
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    # Footprint, to get volume and shot noise; provided nbar is in deg^(-2) dz^(-1)
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances.append(precision.to_covariance(params=['qiso']))

# Let's compare against reference: within a few percent, not too bad!
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([cov.std(params='qiso') for cov in covariances]) / np.array(refs)[:len(covariances)])

[000000.58] [0/1] 01-31 15:28  absl                      INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
[000000.58] [0/1] 01-31 15:28  absl                      INFO     Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.58] [0/1] 01-31 15:28  absl                      INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.60] [0/1] 01-31 15:28  absl                      INFO     Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
[000000.61] [0/1] 01-31 15:28  absl                      WARNING  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


/home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


[000001.59] [0/1] 01-31 15:28  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000001.71] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000001.73] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000001.76] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000001.78] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000001.81] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000001.83] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000001.86] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[

[000008.86] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000008.89] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000008.91] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000008.94] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000008.96] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000008.98] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000009.01] [0/1] 01-31 15:28  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000009.05] [0/1] 01-31 15:28  Differentiation           INFO     Using finite-differentiation for parameter 

[000014.20] [0/1] 01-31 15:28  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000014.27] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000014.33] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000014.40] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000014.48] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000014.58] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[000014.65] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000014.71] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000014.78] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differenti

### Comparison against multipole compression

In [4]:
covariances2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                         # fit monopole and quadrupole, between 0.02 and 0.5 h/Mpc, with 0.005 h/Mpc steps
                                                         klim={0: [0.01, 0.5, 0.005], 2: [0.01, 0.5, 0.005], 4: [0.01, 0.5, 0.005]},
                                                         theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances2.append(precision.to_covariance(params=['qiso']))

# multipole compression is very close to optimal...
print(np.array([cov.std(params='qiso') for cov in covariances2]) / np.array([cov.std(params='qiso') for cov in covariances]))

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


[000023.98] [0/1] 01-31 15:28  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000024.03] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000024.07] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000024.11] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000024.15] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000024.18] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000024.21] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000024.25] [0/1] 01-31 15:28  Differentiation           INFO     U

[000037.75] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000037.80] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000037.84] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000037.89] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000037.95] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000038.00] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000038.05] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al4_-3.
[000038.10] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differentiation for parameter al4_-2.
[000038.15] [0/1] 01-31 15:28  Differentiation           INFO     Using auto-differ

[000056.67] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al4_-1.
[000056.71] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al4_0.
[000056.74] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al4_1.
[000056.77] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000056.80] [0/1] 01-31 15:29  Differentiation           INFO     Using finite-differentiation for parameter qiso.
[000056.84] [0/1] 01-31 15:29  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000056.88] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000056.93] [0/1] 01-31 15:29  Differentiation           INFO    

## BAO + full shape forecasts

In [5]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances_bao, covariances_bao_highk, covariances_fs_lowk, covariances_bao_fs = [], [], [], []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    #for param in theory.params.select(basename='al*'): param.update(fixed=True)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    likelihood.init.update(klim=(0.1, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao_highk.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    #likelihood.init.update(klim=(0.01, 0.1))
    params = {'b1': b1}
    template = StandardPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.1))
    fisher = Fisher(likelihood)
    precision_fs = fisher(**params)
    covariances_fs_lowk.append(precision_fs.to_covariance(params=['qpar', 'qper']))
    covariances_bao_fs.append((precision_bao + precision_fs).to_covariance(params=['qpar', 'qper', 'df']))

[000068.85] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000068.90] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000068.94] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000068.98] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000069.01] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000069.05] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000069.09] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000069.12] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000075.11] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000075.14] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000075.23] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000077.49] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000077.53] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000077.56] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000077.59] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000077.61] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000077.6

[000081.63] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'df', 'b1', 'sn0'].
[000081.69] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000081.73] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000081.77] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000081.80] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000081.84] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000082.53] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000082.58] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.


[000086.73] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000086.75] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000086.77] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000086.79] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'df', 'b1', 'sn0'].
[000086.84] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000086.89] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000086.92] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000086.95] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000086.99] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for

[000092.00] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'df', 'b1', 'sn0'].
[000092.02] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000092.04] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000092.05] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000092.07] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000092.09] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter sn0.
[000092.11] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'df', 'b1', 'sn0'].
[000092.16] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000092.20] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differ

[000096.02] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000096.02] [0/1] 01-31 15:29  Differentiation           INFO     qpar grid is [0.998 1.    1.002].
[000096.02] [0/1] 01-31 15:29  Differentiation           INFO     qper grid is [0.99942265 1.         1.00057735].
[000097.10] [0/1] 01-31 15:29  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'df', 'b1', 'sn0'].
[000097.12] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000097.13] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000097.15] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter df.
[000097.16] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000097.18] [0/1] 01-31 15:29  Differentiation           INFO     Using auto-differentiation for para

In [6]:
# 5% - 15% improvement using the full shape < 0.1 h/Mpc
for cov_bao, cov_bao_fs in zip(covariances_bao, covariances_bao_fs):
    print(cov_bao.std('qpar') / cov_bao_fs.std('qpar'), cov_bao.std('qper') / cov_bao_fs.std('qper'))

1.1579387550689182 1.0708940048279647
1.1596359427242826 1.071301108134669
1.162620481346934 1.0724623344054003
1.168323144909373 1.0753007363189548
1.207185984945957 1.1002945571350582
